In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
curveballDL = pd.read_csv('DraftLeagueCurveball.csv')
curveballMLB = pd.read_csv('MLBcurveball.csv')

In [3]:
curveballDL = curveballDL.astype(str)
curveballMLB = curveballMLB.astype(str)

In [4]:
curveballDL['league'] = 'Draft League'
curveballMLB['league'] = 'MLB'

In [5]:
pitchersDL = list(set(curveballDL['pitcher']))

In [6]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [7]:
def get_index_from_pitcher(pitcher):
    return curveball[curveball.pitcher == pitcher].index[0]

In [8]:
def get_pitcher_from_index(index):
    return curveball[curveball.index == index].values[0][0]

In [9]:
for x in pitchersDL:
    
    try:

        curveball = pd.concat([curveballDL, curveballMLB])

        curveball["combined_features"] = curveball.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(curveball["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        curveball = curveball[(curveball['pitcher'] == pitcher_like) | (curveball['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        i=0

        for pitcher in sorted_similar_pitchers:
            print(get_pitcher_from_index(pitcher[0]),", Cosine Sim: ", sorted_similar_pitchers[i][1])
            i=i+1
            if i>1:
                break
                
    except IndexError:
        print(x, ', No Similarity')
        pass

Fluno, Carson , Cosine Sim:  1.0000000000000002
Slater, Austin , Cosine Sim:  0.2721655269759087
Murphy, Shane , Cosine Sim:  1.0000000000000002
Murphy, Shane , No Similarity
Gilchrist, Malcolm , Cosine Sim:  1.0000000000000002
Lauer, Eric , Cosine Sim:  0.3333333333333333
Escano, Antonio , Cosine Sim:  0.9999999999999999
Escano, Antonio , No Similarity
Patterson, Kyler , Cosine Sim:  1.0000000000000002
Jefferies, Daulton , Cosine Sim:  0.28867513459481287
Minton, Hayden , Cosine Sim:  0.9999999999999999
Hernández, Carlos , Cosine Sim:  0.38138503569823695
Brandon, Connor , Cosine Sim:  0.9999999999999999
Brandon, Connor , No Similarity
Czerniawski, Zachary , Cosine Sim:  1.0000000000000002
Adams, Chance , Cosine Sim:  0.3333333333333333
Sikorski, Easton , Cosine Sim:  1.0000000000000002
Gonzales, Marco , Cosine Sim:  0.31622776601683794
Figueroa, Ryan , Cosine Sim:  0.9999999999999999
Figueroa, Ryan , No Similarity
Pickell, Cannon , Cosine Sim:  1.0000000000000002
Bourque, James , Cos

Cespedes, Angel , Cosine Sim:  0.9999999999999999
Krol, Ian , Cosine Sim:  0.33806170189140666
Alba, Miguel , Cosine Sim:  1.0000000000000002
Kelley, Trevor , Cosine Sim:  0.3333333333333333
Bortka, Josh , Cosine Sim:  0.9999999999999999
Smith, Josh A. , Cosine Sim:  0.30000000000000004
Gowens, Riley , Cosine Sim:  1.0000000000000002
Peterson, David , Cosine Sim:  0.31622776601683794
Bosch, Carter , Cosine Sim:  1.0000000000000002
Ryan, Kyle , Cosine Sim:  0.2721655269759087
Drake, Kohl , Cosine Sim:  1.0000000000000002
Lauer, Eric , Cosine Sim:  0.3333333333333333
Stevens, Will , Cosine Sim:  0.9999999999999999
Difo, Wilmer , Cosine Sim:  0.4216370213557839
Becker, Austin , Cosine Sim:  0.9999999999999999
Bednar, David , Cosine Sim:  0.31622776601683794
Yamane, Tiaga , Cosine Sim:  0.9999999999999999
Difo, Wilmer , Cosine Sim:  0.4216370213557839
Crawford, Devin , Cosine Sim:  1.0000000000000002
Crawford, Devin , No Similarity
Fruit, Zach , Cosine Sim:  1.0000000000000002
Mitchell, Br